# Programme data

This notebook prepares data for the Programme theme page.

In [ ]:
import json
from datetime import date

from utils.paths import SITE
from utils.themes.programme_slice import ProgrammeSlice

In [ ]:
EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

## Events data

Read events

In [ ]:
programme_data = ProgrammeSlice(date_range=(date.min, date.today()))

Create an aggregate by month of the events

In [ ]:
(
    programme_data.events
    .aggregate('month', {
        'Events': ('events', sum),
        'Audience': ('audience', sum),
        'Participants': ('participants', sum),
        # 'Records': (len),
    })
    .convert('month', lambda f: f.isoformat())
    .tocsv(EVENTS / 'total_by_month.csv')
)

In [ ]:
source_values = (
    programme_data.events
    .cut('project_name', 'source', 'events', 'audience', 'participants')
    .melt(['project_name', 'source'])
    .aggregate(['variable', 'project_name', 'source'], sum, 'value')
    .recast(variablefield='source')
    .replaceall(None, 0)
    .addfield('TOTAL', lambda r: sum(c for c in r if type(c) is int))
    .selectgt('TOTAL', 0)
    # .addfield('values', lambda r: {k: r[k] for k in r.flds if k not in ['variable', 'project_name']})
    # .aggregate(['variable', 'project_name'], list, 'values')
    .facet('variable')
)

In [ ]:
# TODO work out how to present this!
{
    'audience': {e['project_name']: {
        p: e[p]
        for p
        in e
        if
            p not in ['project_name']
            and
            # e[p] is not None
            e[p] > 0
    } for e in source_values['audience'].sort('TOTAL', reverse=True).cutout('variable').dicts()}
}

Aggregate by Project and by Month, and convert months to columns

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project.csv')
)

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'audience')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project_audience.csv')
)

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'participants')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project_participants.csv')
)

Aggregate by Project and by Month, and convert projects to columns

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='month', variablefield='project_name', missing=0)
    .tocsv(EVENTS / 'monthly_breakdown.csv')
)

## Project summaries

Create a project breakdown

In [ ]:
project_details = (
    programme_data.project_breakdown
    .replace('evaluation_category', None, 'UNKNOWN')
    .replaceall(None, 0)
    .aggregate(
        'project_name',
        {
            f: (f, sum) for f in ['events', 'audience', 'participants']
        } |
        {
            'evaluation_category': ('evaluation_category', lambda x: ', '.join({i for i in x})),
        }
    )
    .leftjoin(
        programme_data.project_data.aggregate('project_name', {
            'start_date': ('start_date', min),
            'end_date': ('end_date', max),
        })
    )
    .addfield('Details', lambda r: {
        # 'records': r.Records,
        'events': r.events,
        'audience': r.audience,
        'participants': r.participants,

        'evaluationCategory': r.evaluation_category,
        'earliestDate': r.start_date.isoformat() if r.start_date else r.date.isoformat() if r.date else None,
        'latestDate': r.end_date.isoformat() if r.end_date else None,
    })
    .cut('project_name', 'Details')
    .sort('project_name')
)

In [ ]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(
            project_details.records()
        ),
        f,
        indent=2,
    )

Create a summary file

In [ ]:
programme_data.excluded_events_data

In [ ]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': {
                'events': sum(programme_data.events.values('events')),
                'audience': sum(a for a in programme_data.events.values('audience') if a is not None),
                'participants': sum(filter(None.__ne__, programme_data.events.values('participants')), 0),
            },
            'excluded': dict(programme_data.excluded_events_data.aggregate('validation', len).records()),
            'date': {
                'earliest': min(programme_data.events_data.values('start_date')).isoformat(),
                'latest': max(programme_data.events_data.values('end_date')).isoformat(),
            }
        },
        f,
        indent=2,
    )

## Venues

In [ ]:
from utils.themes.programme import Programme

In [ ]:
Programme.venues

In [ ]:
education_settings = (
    Programme.venues
    .selectcontains('organisation_or_venue_type', 'Education Setting')
)

In [ ]:
loading_bay = (
    Programme.venues
    .selectcontains('organisation_or_venue_name', 'Loading Bay')
    .addfield('event_report_count', lambda r: len(r['event_reports']))
    .aggregate(['organisation_or_venue_name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
loading_bay

In [ ]:
beacon = (
    Programme.venues
    .selectcontains('organisation_or_venue_name', 'Beacon - ')
    .addfield('event_report_count', lambda r: len(r['event_reports'] or []))
    .aggregate(['organisation_or_venue_name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
beacon

In [ ]:
json.dump(
    {
        'loading_bay': dict(loading_bay.records()),
        'beacon': dict(beacon.records()),
    },
    open(EVENTS / 'by_venue.json', 'w')
)